# Classification
Classify malware samples according to their labels using their reduced representations.

In [1]:
import tensorflow as tf
print(f"GPU is {'not ' if len(tf.config.list_physical_devices('GPU')) == 0 else ''}available")

2024-04-29 14:11:14.418785: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 14:11:14.956355: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GPU is available


2024-04-29 14:11:15.673632: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 14:11:15.705398: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 14:11:15.705461: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
import keras

# Data Processing

The `1-Data-Exploration.ipynb` file should have been run before this, and, as a result, the `processed-data.csv` file should have been generated.

In [3]:
import pandas as pd

In [4]:
raw_df = pd.read_csv("../data/processed-data.csv")
raw_df

,label,hash,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,f16631469eb35406ef4049d30c763cadda571b25bbdb45...,5.351691,21.723158,27.772293,0.000000,0.0,0.000000,0.128761,14.052113,...,0.0,7.552395,0.00000,0.000000,2.221274,0.0,0.000000,4.876917,7.973024,0.000000
1,DARKKOMET,d31a7102cbc54447c251ba62760eb484fd0c9fbb8ea54f...,0.000000,11.434434,8.999536,0.000000,0.0,0.000000,14.130379,0.000000,...,0.0,0.000000,0.00000,0.000000,0.000000,0.0,11.259565,0.000000,9.895402,0.000000
2,HIVE,122e397dc3a55143bd276d6ff3bc04a05601fbf390aa52...,12.302238,13.320964,10.538460,0.000000,0.0,3.827004,4.751883,11.345034,...,0.0,2.080284,0.00000,0.000000,7.952750,0.0,0.000000,8.721115,0.000000,6.641961
3,AGENTB,b390756adcaa12f5abf6c97174d376027a85efd7606ee5...,22.723690,8.872649,27.881690,0.000000,0.0,0.000000,1.459226,15.731002,...,0.0,6.437061,0.00000,0.000000,1.009103,0.0,0.000000,9.150738,0.000000,8.346413
4,RELINE,f4580de8ed4b87729112d8e7779926905ae7b859dbe4ec...,3.047245,18.274752,9.910273,0.000000,0.0,10.917514,14.134702,0.682731,...,0.0,7.659189,3.30271,5.981354,9.993090,0.0,0.000000,4.339109,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,FARFLI,de965e33dff58cf011106feacef2f804d9e35d00b8b5ff...,8.136786,14.836237,9.742967,0.000000,0.0,10.367283,3.756552,7.201270,...,0.0,2.831204,0.00000,1.749087,9.433194,0.0,0.000000,6.312644,0.000000,0.000000
664,TRICKBOT,9506421d996290f70689559ee0c09cc074c948fff49547...,0.000000,9.550467,8.573158,1.349316,0.0,0.000000,0.000000,2.296499,...,0.0,0.000000,0.00000,0.000000,1.292367,0.0,5.604953,15.579664,23.591751,0.000000
665,NECONYD,0eee965f286f057a3175797590795bbf99fda65dc8d845...,21.578440,5.653350,22.406902,0.000000,0.0,0.000000,8.788987,12.373772,...,0.0,13.251783,0.00000,0.410581,5.572082,0.0,0.000000,4.414941,0.000000,4.865883
666,TRICKBOT,0c6aa0ae05d5fa8bf5a8ea95310be73ee60e55a0ce6864...,0.623411,14.254701,7.313779,0.000000,0.0,9.019151,12.874804,0.080667,...,0.0,0.000000,0.00000,3.972777,0.321772,0.0,0.000000,3.342158,0.000000,0.000000


For training the model, we don't need the `hash` of the sample.

In [5]:
df = raw_df.drop(columns=["hash"])
df

,label,dim-00,dim-01,dim-02,dim-03,dim-04,dim-05,dim-06,dim-07,dim-08,...,dim-22,dim-23,dim-24,dim-25,dim-26,dim-27,dim-28,dim-29,dim-30,dim-31
0,TRICKBOT,5.351691,21.723158,27.772293,0.000000,0.0,0.000000,0.128761,14.052113,0.000000,...,0.0,7.552395,0.00000,0.000000,2.221274,0.0,0.000000,4.876917,7.973024,0.000000
1,DARKKOMET,0.000000,11.434434,8.999536,0.000000,0.0,0.000000,14.130379,0.000000,11.213587,...,0.0,0.000000,0.00000,0.000000,0.000000,0.0,11.259565,0.000000,9.895402,0.000000
2,HIVE,12.302238,13.320964,10.538460,0.000000,0.0,3.827004,4.751883,11.345034,9.400712,...,0.0,2.080284,0.00000,0.000000,7.952750,0.0,0.000000,8.721115,0.000000,6.641961
3,AGENTB,22.723690,8.872649,27.881690,0.000000,0.0,0.000000,1.459226,15.731002,5.915589,...,0.0,6.437061,0.00000,0.000000,1.009103,0.0,0.000000,9.150738,0.000000,8.346413
4,RELINE,3.047245,18.274752,9.910273,0.000000,0.0,10.917514,14.134702,0.682731,1.784129,...,0.0,7.659189,3.30271,5.981354,9.993090,0.0,0.000000,4.339109,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,FARFLI,8.136786,14.836237,9.742967,0.000000,0.0,10.367283,3.756552,7.201270,4.494106,...,0.0,2.831204,0.00000,1.749087,9.433194,0.0,0.000000,6.312644,0.000000,0.000000
664,TRICKBOT,0.000000,9.550467,8.573158,1.349316,0.0,0.000000,0.000000,2.296499,0.000000,...,0.0,0.000000,0.00000,0.000000,1.292367,0.0,5.604953,15.579664,23.591751,0.000000
665,NECONYD,21.578440,5.653350,22.406902,0.000000,0.0,0.000000,8.788987,12.373772,10.141718,...,0.0,13.251783,0.00000,0.410581,5.572082,0.0,0.000000,4.414941,0.000000,4.865883
666,TRICKBOT,0.623411,14.254701,7.313779,0.000000,0.0,9.019151,12.874804,0.080667,10.079009,...,0.0,0.000000,0.00000,3.972777,0.321772,0.0,0.000000,3.342158,0.000000,0.000000


Separate into `X` and `y`.

In [6]:
X = df.drop(columns=["label"])
y = df["label"]

In [7]:
print(X.shape)
print(y.shape)

(668, 32)
(668,)


# Label Encoding and Train-Test Split

We need to encode the labels for the `y`.

In [8]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

We want to save the label encoder for use in the full model.

In [9]:
import pickle

with open("../models/classifier/label-encoder.pkl", "wb") as f:
    pickle.dump(label_encoder, f)

80% of the data will be saved for training, while 20% will be left for testing.

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [11]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(534, 32)
(134, 32)
(534,)
(134,)


# Model Creation

Nothing too special here, we just use several dense layers for classification.

In [12]:
from keras import layers
from keras.models import Sequential

In [13]:
INITIAL_EPOCH = 0

def create_model():
    model = Sequential(name="Classifier")

    model.add(keras.Input((X.shape[1],), name="input"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(256, activation="relu"))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(len(label_encoder.classes_), activation="softmax"))

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer="adam",
        metrics=["sparse_categorical_accuracy"]
    )
    return model

In [14]:
model = create_model()
model.summary()

2024-04-29 14:11:16.010149: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 14:11:16.010243: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 14:11:16.010279: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 14:11:16.742753: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-04-29 14:11:16.742823: I external/local_xla/xla/stream_executor

Model: "Classifier"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        16,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 52)             │         3,380 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 38,580 (150.70 KB)

 Trainable params: 38,580 (150.70 KB)

 Non-trainable params: 0 (0.00 B)

Define callbacks.

In [15]:
import os

checkpoint_path = "../models/classifier/checkpoint.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

In [16]:
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, verbose=1, min_delta=1e-4)

Load latest checkpoint if there is one.

In [17]:
# INITIAL_EPOCH = 54
# model = keras.models.load_model(checkpoint_path)

Train the model.

In [18]:
NUM_EPOCHS = 100

model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    initial_epoch=INITIAL_EPOCH,
    epochs=NUM_EPOCHS,
    callbacks=[checkpointer, early_stopping]
)

Epoch 1/100


I0000 00:00:1714371077.673645 1059244 service.cc:145] XLA service 0x7f5984003f60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714371077.673688 1059244 service.cc:153]   StreamExecutor device (0): Quadro P1000, Compute Capability 6.1
2024-04-29 14:11:17.692506: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-29 14:11:18.528653: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 1/14 ━━━━━━━━━━━━━━━━━━━━ 35s 3s/step - loss: 6.8323 - sparse_categorical_accuracy: 0.0000e+00

I0000 00:00:1714371079.760119 1059244 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - loss: 4.8791 - sparse_categorical_accuracy: 0.0514   
Epoch 1: val_loss improved from inf to 3.54976, saving model to ../models/classifier/checkpoint.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 125ms/step - loss: 4.8362 - sparse_categorical_accuracy: 0.0537 - val_loss: 3.5498 - val_sparse_categorical_accuracy: 0.1869
Epoch 2/100
 1/14 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 3.3630 - sparse_categorical_accuracy: 0.2188
Epoch 2: val_loss improved from 3.54976 to 3.13780, saving model to ../models/classifier/checkpoint.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.1827 - sparse_categorical_accuracy: 0.2880 - val_loss: 3.1378 - val_sparse_categorical_accuracy: 0.2430
Epoch 3/100
 1/14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2.7723 - sparse_categorical_accuracy: 0.3438
Epoch 3: val_loss improved from 3.13780 to 2.83186, saving model to ../models/classifier/checkpoint.keras
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 2.6877 - sparse_categoric

Load the best performing model.

In [19]:
model = keras.models.load_model(checkpoint_path)

In [20]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print(f"Testing loss:      {test_loss:5.5f}")
print(f"Testing accuracy:  {test_acc:5.5f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 115ms/step - loss: 1.9782 - sparse_categorical_accuracy: 0.5400
Testing loss:      1.99355
Testing accuracy:  0.55224


In [21]:
model.save("../models/classifier/classifier.keras")